In [1]:
import pandas as pd
import numpy as np
import dias.rewriter

In [2]:
md = pd.read_csv('./datasets/rounakbanik__movie-recommender-systems__movies_metadata.csv', low_memory=False)

# -- STEFANOS -- Replicate Data

In [3]:
factor = 220
md = pd.concat([md]*factor)

# -- STEFANOS -- Prelude

In [4]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.65825

In [5]:
m = vote_counts.quantile(0.95)
m

1190.1999999997206

In [6]:
%%time
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

CPU times: user 1.84 s, sys: 32 ms, total: 1.87 s
Wall time: 1.87 s


In [7]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'vote_count', 'vote_average', 'popularity']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(44000, 4)

# -- STEFANOS -- `apply()` with `axis=1` and only assignments and math

*Original time*: 232ms

*Rewritten*: 1.5ms

## Speedup: 154x

In [8]:
%%time
%%rewrite
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

CPU times: user 2.6 ms, sys: 1.27 ms, total: 3.87 ms
Wall time: 3.47 ms


[IREWRITE JSON]
{
  "raw": "def weighted_rating(x):\n    v = x['vote_count']\n    R = x['vote_average']\n    return (v/(v+m) * R) + (m/(m+v) * C)\nqualified['wr'] = qualified.apply(weighted_rating, axis=1)\n",
  "modified": "def weighted_rating(x):\n    v = x['vote_count']\n    R = x['vote_average']\n    return v / (v + m) * R + m / (m + v) * C\nqualified['wr'] = weighted_rating(qualified)\n",
  "patts-hit": {},
  "rewritten-exec-time": 1.54797
}
[IREWRITE END JSON]
